In [15]:
import surprise
import pandas as pd

In [185]:
# Make mapping dict
details = [x for x in range(101)]
rating = [1] * 25 + [2] * 25 + [3] * 25 + [4] * 25 + [5] * 25
d_map = dict(zip(details, rating))

In [186]:
df = pd.read_csv('C:/Users/emoll/Downloads/out2.csv')
df = df.drop(columns=['Unnamed: 0'])
# SHAPE : UserID, ItemID, progress, Year

In [189]:
df['rating'] = df['progress'].map(d_map)

In [190]:
df.columns = ['userID', 'itemID', 'progress', 'Year', 'rating']

In [191]:
# Split train and test sets.
df_train = df[df.Year < 2022]
print(df_train.shape)
df_train = df_train.groupby(['userID', 'itemID']).agg({'rating': 'max'}).reset_index()
print(df_train.shape)

df_test = df[df.Year == 2022]
df_test = df_test.groupby(['userID', 'itemID']).agg({'rating': 'max'}).reset_index()
# Keep only known users in testset
df_test = df_test[df_test['userID'].isin(list(set(df_train.userID)))]

(7020071, 5)
(3624958, 3)
(3624958, 3)


In [195]:
from surprise import SVD, SVDpp
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, train_test_split

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_train[['userID', 'itemID', 'rating']], reader)
print("making train/test set...")
trainset, testset = train_test_split(data, test_size=.01) #Lost 1% of data. Solution quick and dirty

# Define model
algo = SVD()
print('fit algo...')
# 
algo.fit(trainset)
#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

data...
fit algo...


In [196]:
# Get predictions to make from events existing in testset
preds_to_make = list(zip(df_test.userID, df_test.itemID, df_test.rating))

In [197]:
# Make predictions
outputs = []
for user, item, _rating in preds_to_make:
    pred = algo.predict(user, item, r_ui=_rating, verbose=False)
    
    outputs.append((user, item, _rating, pred.est))


In [198]:
# Compute our poorly performing rmse :)
from sklearn.metrics import mean_squared_error 

realVals = [x[2] for x in outputs]
predictedVals = [x[3] for x in outputs]

rmse = mean_squared_error(realVals, predictedVals, squared=False)
rmse

In [200]:
import numpy as np
np.set_printoptions(suppress=True)

In [201]:
from collections import Counter
d = dict(Counter([(x[2], round(x[3])) for x in outputs]))

In [205]:
# Roughly display real and predicted value matrix
values = [1, 2, 3, 4, 5]

output = np.zeros((len(values), len(values)), dtype=float)
for key in d:
    i = values.index(key[0])
    j = values.index(key[1])
    output[i, j] = d[key]

print(output)

[[33282. 25077.  8185.  2925.   955.]
 [ 2586.  3405.  1299.   475.   158.]
 [  683.  1042.   392.   151.    48.]
 [  461.   682.   305.   128.    46.]
 [ 3964.  7943.  7522.  6909.  3762.]]


In [203]:
sum(sum(output))

112385.0

In [204]:
sum(np.diag(output))

40969.0